In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
data_train = pd.read_csv('data/final/smoothed/data_train.csv')
data_train = data_train.drop(columns="Unnamed: 0")

# Let's implement the thing, where deep OTM, OTM, ATM, ITM, deep ITM is a thing

# we have to discriminate between calls and puts
# Coding; deep OTM = 1, OTM =2, ATM =3, ITM = 4, deep ITM=5 
# outliers, sort of?
print(data_train.shape)
data_train = data_train[data_train['moneyness'] >= 0.8]
data_train = data_train[data_train['moneyness'] <= 1.6]
print(data_train.shape)

data_train.loc[(data_train['cp_flag']=='C') & (data_train['moneyness'] <0.90), 'moneyness_enc'] = 1
data_train.loc[(data_train['cp_flag']=='C') & (data_train['moneyness'] >=0.90) & (data_train['moneyness'] < 0.97), 'moneyness_enc'] = 2
data_train.loc[(data_train['cp_flag']=='C') & (data_train['moneyness'] >=0.97) & (data_train['moneyness'] < 1.03), 'moneyness_enc'] = 3
data_train.loc[(data_train['cp_flag']=='C') & (data_train['moneyness'] >=1.03) & (data_train['moneyness'] < 1.10), 'moneyness_enc'] = 4
data_train.loc[(data_train['cp_flag']=='C') & (data_train['moneyness'] >=1.10), 'moneyness_enc'] = 5

data_train.loc[(data_train['cp_flag']=='P') & (data_train['moneyness'] <0.90), 'moneyness_enc'] = 5
data_train.loc[(data_train['cp_flag']=='P') & (data_train['moneyness'] >=0.90) & (data_train['moneyness'] < 0.97), 'moneyness_enc'] = 4
data_train.loc[(data_train['cp_flag']=='P') & (data_train['moneyness'] >=0.97) & (data_train['moneyness'] < 1.03), 'moneyness_enc'] = 3
data_train.loc[(data_train['cp_flag']=='P') & (data_train['moneyness'] >=1.03) & (data_train['moneyness'] < 1.10), 'moneyness_enc'] = 2
data_train.loc[(data_train['cp_flag']=='P') & (data_train['moneyness'] >=1.10), 'moneyness_enc'] = 1

(580068, 29)
(578724, 29)


In [5]:
print(data_train.head())
print(data_train.shape)

         date               symbol      exdate   last_date cp_flag  \
0  2012-01-03  SPXW 120106C1200000  2012-01-06  03/01/2012       C   
1  2012-01-03  SPXW 120106C1210000  2012-01-06  03/01/2012       C   
2  2012-01-03  SPXW 120106C1220000  2012-01-06  03/01/2012       C   
3  2012-01-03  SPXW 120106C1230000  2012-01-06  03/01/2012       C   
4  2012-01-03  SPXW 120106C1235000  2012-01-06  03/01/2012       C   

   strike_price  best_bid  best_offer  volume  open_interest  ...  \
0          1200      75.8        78.7       2             69  ...   
1          1210      66.0        68.8       1              2  ...   
2          1220      56.1        58.8       1              4  ...   
3          1230      46.2        49.1       1             21  ...   
4          1235      41.4        44.1       1              0  ...   

          High          Low        Close    Adj Close      Volume  moneyness  \
0  1284.619995  1258.859985  1277.060059  1277.060059  3943710000   1.064217   
1  1

In [6]:
data_validation = pd.read_csv("data/final/evaluation/validation_set.csv")

In [36]:
for i in data_train['date'].unique():
    print(i, data_train[data_train['date']==i]['moneyness_enc'].unique(), data_train[data_train['date']==i]['maturity'].unique(), data_train[data_train['date']==i].shape)

2012-01-03 [4. 3. 2.] [3] (49, 30)
2012-01-04 [4. 3. 2.] [2] (35, 30)
2012-01-05 [3. 2. 4.] [1] (28, 30)
2012-01-06 [4. 3. 2. 1.] [5] (60, 30)
2012-01-09 [4. 3. 2.] [4] (43, 30)
2012-01-10 [3. 2. 4.] [3] (43, 30)
2012-01-11 [4. 3. 2.] [2] (39, 30)
2012-01-12 [4. 3.] [1] (27, 30)
2012-01-17 [5. 4. 3. 2. 1.] [3] (77, 30)
2012-01-18 [3. 2. 4.] [2] (44, 30)
2012-01-20 [4. 3. 2. 1.] [5] (66, 30)
2012-01-23 [5. 4. 3. 2. 1.] [4] (58, 30)
2012-01-24 [4. 3. 2.] [3] (47, 30)
2012-01-25 [4. 3. 2.] [2] (43, 30)
2012-01-26 [3. 2. 4.] [1] (26, 30)
2012-01-27 [5. 4. 3. 2. 1.] [5] (68, 30)
2012-01-30 [3. 2. 4.] [4] (50, 30)
2012-01-31 [4. 3. 2.] [3] (47, 30)
2012-02-01 [3. 2. 4.] [2] (38, 30)
2012-02-02 [4. 3.] [1] (27, 30)
2012-02-03 [5. 4. 3. 2. 1.] [5] (64, 30)
2012-02-06 [3. 2.] [4] (46, 30)
2012-02-07 [5. 4. 3. 2.] [3] (55, 30)
2012-02-08 [3. 2. 4.] [2] (36, 30)
2012-02-09 [4. 3. 2.] [1] (27, 30)
2012-02-13 [3. 2. 1. 4.] [4] (55, 30)
2012-02-14 [3. 1. 2. 4.] [3] (48, 30)
2012-02-15 [3. 1. 2. 4.] 

In [7]:
print(data_train[data_train['date']=='2012-01-03'][['strike_price','moneyness', 'moneyness_enc','maturity', 'IV_smooth','impl_volatility']])

    strike_price  moneyness  moneyness_enc  maturity  IV_smooth  \
0           1200   1.064217            4.0         3   0.356446   
1           1210   1.055422            4.0         3   0.333749   
2           1220   1.046771            4.0         3   0.296131   
3           1230   1.038260            4.0         3   0.268058   
4           1235   1.034057            4.0         3   0.251624   
5           1240   1.029887            3.0         3   0.239309   
6           1245   1.025751            3.0         3   0.235642   
7           1250   1.021648            3.0         3   0.224416   
8           1255   1.017578            3.0         3   0.220456   
9           1260   1.013540            3.0         3   0.188592   
10          1265   1.009534            3.0         3   0.193897   
11          1270   1.005559            3.0         3   0.182288   
12          1275   1.001616            3.0         3   0.179902   
13          1280   0.997703            3.0         3   0.17924

In [8]:
#So, we have multiple observations per moneyness_enc, and max 5 maturities. 
#Not every date has the same number of observations, and that is a problem.
#What can we do to fix it? -> Make number of observations the same 
#Get maximum number of observations first

max_rows = 0
max_date = 0
for i in data_train['date'].unique():

    if data_train[data_train['date']==i].shape[0] > max_rows:
        max_date = i
        max_rows = data_train[data_train['date']==i].shape[0]
    
print(max_date, max_rows)

2020-03-25 1254


In [9]:
pd.set_option('display.max_row', None)
cols = ['strike_price', 'moneyness', 'moneyness_enc', 'maturity', 'IV_smooth']
print(data_train[data_train['date']==max_date][cols])

        strike_price  moneyness  moneyness_enc  maturity  IV_smooth
334596          2140   1.156804            5.0         2   0.834707
334597          2145   1.154107            5.0         2   0.845780
334598          2150   1.151423            5.0         2   0.856358
334599          2160   1.146093            5.0         2   0.876083
334600          2170   1.140811            5.0         2   0.893980
334601          2180   1.135578            5.0         2   0.910147
334602          2200   1.125255            5.0         2   0.937652
334603          2220   1.115117            5.0         2   0.959289
334604          2225   1.112611            5.0         2   0.963857
334605          2230   1.110117            5.0         2   0.968108
334606          2235   1.107633            5.0         2   0.972050
334607          2240   1.105161            5.0         2   0.975694
334608          2245   1.102699            5.0         2   0.979047
334609          2250   1.100249            5.0  

In [45]:
# Function to pad each date's observations
def pad_observations(df, max_obs):
    padded_data = np.full((max_obs, 3), np.nan)  # (moneyness, maturity, IV)
    obs = df[['moneyness_enc', 'maturity', 'IV_smooth']].values
    padded_data[:len(obs)] = obs  # Fill available data
    return np.nan_to_num(padded_data)  # Convert NaNs to 0

# Apply padding
X_padded = np.array([pad_observations(data_train[data_train['date'] == d], max_rows) for d in data_train['date'].unique()])

# print(X_padded.groupby('date').size())
# Mask padded zeros in the model
# from tensorflow.keras.layers import Masking
# model.add(Masking(mask_value=0, input_shape=(max_obs, 3)))  # Ignore padded values

AttributeError: 'numpy.ndarray' object has no attribute 'groupby'

In [47]:
print(X_padded.shape)

(2406, 1254, 3)


In [48]:
print(len(data_train['date'].unique()))

2406


In [16]:
print(data_train['moneyness'])

0         1.064217
1         1.055422
2         1.046771
3         1.038260
4         1.034057
            ...   
580063    0.964637
580064    0.962614
580065    0.961606
580066    0.952629
580067    0.948692
Name: moneyness, Length: 580068, dtype: float64


In [ ]:
# First issue: the number of grids per datatime is not even across the entire sample.
# Use zero padding, or consider interpolation?
# Only a few missing points -> padding is preferred
# If the model can ignore zeros (convLSTM with batch norm -> padding is okay)

# Pay attention to the input layer dimensions
# We want like
# X.shape = (samples, 10, 10, 10, 1)  # 10 days of IV surfaces (input)
# y.shape = (samples, 10, 10, 1)    
# # Notice the extra dimension in X   

In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Conv2D

# Define a ConvLSTM2D model
model = Sequential([
    ConvLSTM2D(
        filters=16, kernel_size=(3, 3),
        activation='relu',
        input_shape=(10, 64, 64, 1),  # (time_steps, rows, cols, channels)
        return_sequences=False
    ),
    BatchNormalization(),
    Conv2D(filters=1, kernel_size=(3, 3), activation='sigmoid', padding='same')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy')

# Print the model summary
model.summary()

# Example input data (batch_size, time_steps, rows, cols, channels)
batch_size = 2
time_steps = 10
height = 64
width = 64
channels = 1

# Generate random input data
example_input = np.random.rand(batch_size, time_steps, height, width, channels).astype(np.float32)

# Example prediction
example_output = model.predict(example_input)

print("Example Input Shape:", example_input.shape)
print("Example Output Shape:", example_output.shape)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_lstm2d_4 (ConvLSTM2D)      │ (None, 62, 62, 16)     │         9,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 62, 62, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 62, 62, 1)      │           145 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,065 (39.32 KB)

 Trainable params: 10,033 (39.19 KB)

 Non-trainable params: 32 (128.00 B)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 645ms/step
Example Input Shape: (2, 10, 64, 64, 1)
Example Output Shape: (2, 62, 62, 1)


In [12]:
print(data_train.columns)

Index(['Unnamed: 0', 'date', 'symbol', 'exdate', 'last_date', 'cp_flag',
       'strike_price', 'best_bid', 'best_offer', 'volume', 'open_interest',
       'impl_volatility', 'delta', 'gamma', 'vega', 'theta', 'optionid',
       'am_settlement', 'expiry_indicator', 'maturity', 'Open', 'High', 'Low',
       'Close', 'Adj Close', 'Volume', 'moneyness', 'midpoint', 'year',
       'IV_smooth', 'moneyness_enc'],
      dtype='object')


In [10]:
print(example_input)

[[[[[0.00441805]
    [0.78462243]
    [0.60830384]
    ...
    [0.8264866 ]
    [0.27865714]
    [0.35727894]]

   [[0.55810326]
    [0.54457855]
    [0.44118223]
    ...
    [0.9903226 ]
    [0.10868084]
    [0.25383604]]

   [[0.3584498 ]
    [0.6987579 ]
    [0.38816163]
    ...
    [0.805965  ]
    [0.71614414]
    [0.23861682]]

   ...

   [[0.41840658]
    [0.32790056]
    [0.20211615]
    ...
    [0.71419924]
    [0.9230194 ]
    [0.5695349 ]]

   [[0.49252936]
    [0.07024787]
    [0.08945027]
    ...
    [0.4087067 ]
    [0.7235932 ]
    [0.2812814 ]]

   [[0.84742033]
    [0.8141357 ]
    [0.5402433 ]
    ...
    [0.20242812]
    [0.13229786]
    [0.46143883]]]


  [[[0.2166206 ]
    [0.02787942]
    [0.38738784]
    ...
    [0.07041775]
    [0.69705176]
    [0.68910635]]

   [[0.22827587]
    [0.6235555 ]
    [0.21824725]
    ...
    [0.03085022]
    [0.13360499]
    [0.7088784 ]]

   [[0.80750585]
    [0.94527465]
    [0.5362307 ]
    ...
    [0.5998129 ]
    [0.5475299 ]
 

In [11]:
print(example_output)

[[[[0.5200112 ]
   [0.5142514 ]
   [0.50893986]
   ...
   [0.5070945 ]
   [0.50442874]
   [0.49356198]]

  [[0.5176684 ]
   [0.5225671 ]
   [0.50890046]
   ...
   [0.5137171 ]
   [0.50533587]
   [0.49901468]]

  [[0.5080214 ]
   [0.5169725 ]
   [0.5121117 ]
   ...
   [0.51041186]
   [0.5154468 ]
   [0.5011171 ]]

  ...

  [[0.5094288 ]
   [0.5166764 ]
   [0.5148639 ]
   ...
   [0.508696  ]
   [0.5224623 ]
   [0.5026477 ]]

  [[0.51704293]
   [0.5155296 ]
   [0.5167592 ]
   ...
   [0.517774  ]
   [0.51942974]
   [0.5036689 ]]

  [[0.5191037 ]
   [0.51870084]
   [0.51647514]
   ...
   [0.51629984]
   [0.5184979 ]
   [0.5105654 ]]]


 [[[0.52510226]
   [0.519394  ]
   [0.5142109 ]
   ...
   [0.507844  ]
   [0.5052271 ]
   [0.49503452]]

  [[0.5154486 ]
   [0.5162368 ]
   [0.512833  ]
   ...
   [0.5028558 ]
   [0.4997356 ]
   [0.5020287 ]]

  [[0.5162291 ]
   [0.5185796 ]
   [0.5141234 ]
   ...
   [0.51888096]
   [0.51447403]
   [0.50160295]]

  ...

  [[0.5226729 ]
   [0.5221958 ]
   [0.5

In [5]:
print(data_train.columns)

Index(['Unnamed: 0', 'date', 'symbol', 'exdate', 'last_date', 'cp_flag',
       'strike_price', 'best_bid', 'best_offer', 'volume', 'open_interest',
       'impl_volatility', 'delta', 'gamma', 'vega', 'theta', 'optionid',
       'am_settlement', 'expiry_indicator', 'maturity', 'Open', 'High', 'Low',
       'Close', 'Adj Close', 'Volume', 'moneyness', 'midpoint', 'year',
       'IV_smooth'],
      dtype='object')


In [6]:
print(data_train['IV_smooth'].head())

0    0.356446
1    0.333749
2    0.296131
3    0.268058
4    0.251624
Name: IV_smooth, dtype: float64


In [ ]:
# amount that is ITM
print(data_train[(data_train['cp_flag']=='C') & (data_train['moneyness']> 1.03)].shape)
print(data_train[(data_train['cp_flag']=='P') & (data_train['moneyness']< 0.97)].shape)

(45627, 30)
(16966, 30)


In [ ]:
# small percentage, lets not drop them
# quick investigation about the amount of puts and calls

print(data_train[data_train['cp_flag']=='C'].shape)
print(data_train[data_train['cp_flag']=='P'].shape)
# Can can run one too, with both of them.. calls would be hindered more than puts



(223649, 30)
(356419, 30)


In [4]:
# Drop the observations outside of the range.. 
# outliers, sort of?
print(data_train.shape)
data_train = data_train[data_train['moneyness'] >= 0.8]
data_train = data_train[data_train['moneyness'] <= 1.6]
print(data_train.shape)

(580068, 30)
(578724, 30)


In [5]:
# # Let's implement the thing, where deep OTM, OTM, ATM, ITM, deep ITM is a thing

# # we have to discriminate between calls and puts
# # Coding; deep OTM = 1, OTM =2, ATM =3, ITM = 4, deep ITM=5 


# data_train.loc[(data_train['cp_flag']=='C') & (data_train['moneyness'] <0.90), 'moneyness_enc'] = 1
# data_train.loc[(data_train['cp_flag']=='C') & (data_train['moneyness'] >=0.90) & (data_train['moneyness'] < 0.97), 'moneyness_enc'] = 2
# data_train.loc[(data_train['cp_flag']=='C') & (data_train['moneyness'] >=0.97) & (data_train['moneyness'] < 1.03), 'moneyness_enc'] = 3
# data_train.loc[(data_train['cp_flag']=='C') & (data_train['moneyness'] >=1.03) & (data_train['moneyness'] < 1.10), 'moneyness_enc'] = 4
# data_train.loc[(data_train['cp_flag']=='C') & (data_train['moneyness'] >=1.10), 'moneyness_enc'] = 5

# data_train.loc[(data_train['cp_flag']=='P') & (data_train['moneyness'] <0.90), 'moneyness_enc'] = 5
# data_train.loc[(data_train['cp_flag']=='P') & (data_train['moneyness'] >=0.90) & (data_train['moneyness'] < 0.97), 'moneyness_enc'] = 4
# data_train.loc[(data_train['cp_flag']=='P') & (data_train['moneyness'] >=0.97) & (data_train['moneyness'] < 1.03), 'moneyness_enc'] = 3
# data_train.loc[(data_train['cp_flag']=='P') & (data_train['moneyness'] >=1.03) & (data_train['moneyness'] < 1.10), 'moneyness_enc'] = 2
# data_train.loc[(data_train['cp_flag']=='P') & (data_train['moneyness'] >=1.10), 'moneyness_enc'] = 1

In [28]:
print(data_train.shape)

(578724, 31)


In [14]:
pd.set_option('display.max_row',None)
print(data_train[data_train['moneyness']<0.8].shape)
print(data_train[data_train['moneyness']<0.8][['date','moneyness', 'cp_flag']])

(1079, 30)
              date  moneyness cp_flag
23246   2014-01-13   0.606400       P
37154   2014-12-17   0.670963       P
57623   2015-12-16   0.592306       P
93629   2016-12-12   0.752320       P
93630   2016-12-12   0.644846       P
93878   2016-12-13   0.649063       P
105939  2017-03-15   0.795087       P
147763  2018-01-12   0.796069       P
207757  2018-10-16   0.780533       P
218237  2018-11-19   0.727224       P
219573  2018-11-26   0.786309       P
221789  2018-12-04   0.794135       P
232709  2019-01-10   0.741897       P
234130  2019-01-16   0.792758       P
235306  2019-01-23   0.694395       P
235555  2019-01-24   0.695350       P
236161  2019-01-25   0.701253       P
237740  2019-02-01   0.751814       P
238419  2019-02-07   0.751681       P
239216  2019-02-11   0.752722       P
240362  2019-02-15   0.771000       P
242139  2019-02-27   0.775661       P
244698  2019-03-12   0.697880       P
246105  2019-03-19   0.786825       P
247572  2019-03-27   0.719326       P
2

In [15]:
print(data_train[data_train['moneyness']>1.6].shape)

print(data_train[data_train['moneyness']>1.6][['date','moneyness', 'cp_flag']])

(265, 30)
              date  moneyness cp_flag
16345   2013-07-15   1.682500       C
18675   2013-09-25   1.781863       C
21884   2013-12-16   1.786540       C
30043   2014-07-15   1.644400       C
38438   2015-01-12   1.655722       C
38739   2015-01-16   2.243800       C
48949   2015-08-21   1.677353       P
48950   2015-08-21   1.642408       P
48951   2015-08-21   1.608890       P
49143   2015-08-24   1.893210       P
49144   2015-08-24   1.721100       P
49145   2015-08-24   1.646270       P
49248   2015-08-24   1.992853       P
51714   2015-09-15   1.683481       C
57282   2015-12-14   2.021940       C
57283   2015-12-14   1.838127       C
63582   2016-03-11   2.022190       C
63583   2016-03-11   1.617752       C
71997   2016-06-14   1.660256       C
81233   2016-09-09   2.300335       C
81307   2016-09-09   2.127810       C
85741   2016-10-14   2.132980       C
105590  2017-03-14   1.971208       C
127987  2017-08-24   1.625980       C
129055  2017-09-01   1.651033       C
13

In [ ]:
# Moneyness was not processed properly
# follow the same approach as in literature...
# Drop 0.8< and >1.6
# five categories, but distinct for both call and put options;
# deep OTM, OTM, ATM, ITM, deep ITM. order flips for both
# write to csv 

In [21]:

def create_model(n_params, 
                 dropout, 
                 recurrent_dropout, 
                 n_convlstm_layers = 2,
                 hidden_activation =  tf.keras.activations.tanh, 
                 optimizer = keras.optimizers.Adam()):

    # input layer
    input_layer = layers.Input(shape= (None,5,5,1) )
    
    # lstm layers
    lstm = input_layer
    for i in range( n_convlstm_layers ):
        lstm =  layers.ConvLSTM2D( 
            kernel_size= (1,1), 
            filters=n_params, 
            data_format= 'channels_last', 
            return_sequences = i<n_convlstm_layers-1,
            activation=hidden_activation,
            padding = "same",
            dropout=dropout, 
            recurrent_dropout=recurrent_dropout
        )( lstm )
        lstm = layers.BatchNormalization()(lstm)    

    output = layers.Conv2D(
        filters=1, kernel_size=(1, 1), activation="linear", padding="same"
    )( lstm )
    output_layer = layers.Reshape((5,5))(output)

    # compile
    model = models.Model( input_layer, output_layer )
    model.compile(
        loss= "MAE",
        optimizer=optimizer, 
    ) 
    
    print(model.summary())
    return model

def train_model(model, 
                x_train, 
                y_train,
                verbose = True, 
                save : "dir" = False,
                training_kwarg_overwrites : "dict" = {} ):
    
    # train until we run out of improvement
    callbacks = [
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=15),
    ]
    
    # train model
    training_kwargs = {
        "x" : x_train,
        "y" : y_train, #dataset["train"]["y_scaled"],
        "epochs" : 200,
        "batch_size" : 64,
        "verbose" : verbose,
        "validation_split" : 0.2,
        "callbacks" : callbacks,
    } 
    training_kwargs.update(training_kwarg_overwrites)
    train_hist = model.fit( **training_kwargs )
    
    
    if save:
        Path(save).mkdir(parents=True, exist_ok=True) # make a home for the models
        train_start, train_end = [ f( dataset["dates"]["train"] ) for f in (min,max) ]
        model_name = "-".join( date.strftime("%Y%m%d") for date in [train_start, train_end] )
        model.save( save+model_name )
        
    return model, train_hist

model = create_model(n_params=10,dropout=0.1,recurrent_dropout=0.1,n_convlstm_layers=2)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, None, 5, 5, 1)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_2 (ConvLSTM2D)      │ (None, None, 5, 5, 10) │           480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, None, 5, 5, 10) │            40 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_3 (ConvLSTM2D)      │ (None, 5, 5, 10)       │           840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 5, 5, 10)       │            40 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 5, 5, 1)        │            11 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 5, 5)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,411 (5.51 KB)

 Trainable params: 1,371 (5.36 KB)

 Non-trainable params: 40 (160.00 B)

None


In [22]:
print(data_train.columns.tolist())

['date', 'symbol', 'exdate', 'last_date', 'cp_flag', 'strike_price', 'best_bid', 'best_offer', 'volume', 'open_interest', 'impl_volatility', 'delta', 'gamma', 'vega', 'theta', 'optionid', 'am_settlement', 'expiry_indicator', 'maturity', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'moneyness', 'midpoint', 'year', 'IV_smooth']


In [ ]:
train_model(model=model, x_train=data_train[['']])

In [17]:
import numpy as np

# Define dataset parameters
num_samples = 1000  # Number of training examples
time_steps = 10      # Length of time series (e.g., 10 past time steps)
height, width = 5, 5 # Grid dimensions
channels = 1         # Only 1 feature per grid cell

# Generate random training data
X_train = np.random.rand(num_samples, time_steps, height, width, channels).astype(np.float32) 
Y_train = np.random.rand(num_samples, height, width).astype(np.float32)  # No time step in output

# Validation data (for evaluation)
X_val = np.random.rand(200, time_steps, height, width, channels).astype(np.float32)
Y_val = np.random.rand(200, height, width).astype(np.float32)


In [ ]:
# check out functional or sequential, and also, what dimension x should be,
# and, how time is shown..

In [7]:
# Create a ConvLSTM2D model

model = models.Sequential([
    layers.ConvLSTM2D(
        filters=32,  # Number of convolutional filters
        kernel_size=(3, 3),  # Height and width of the convolution window
        input_shape=(10, 64, 64, 1),  # (time_steps, height, width, channels)
        padding='same',
        return_sequences=True,  # Return full sequence
        activation='relu'
    ),
    layers.BatchNormalization(),  # Normalize intermediate outputs
    layers.ConvLSTM2D(
        filters=64,
        kernel_size=(3, 3),
        padding='same',
        return_sequences=False,  # Only return the last output
        activation='relu'
    ),
    layers.BatchNormalization(),
    layers.Conv2D(
        filters=1,  # Final output has 1 channel
        kernel_size=(3, 3),
        activation='sigmoid',  # Adjust based on your task
        padding='same'
    )
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


c:\Users\hydra\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_lstm2d_2 (ConvLSTM2D)      │ (None, 10, 64, 64, 32) │        38,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 10, 64, 64, 32) │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_3 (ConvLSTM2D)      │ (None, 64, 64, 64)     │       221,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 1)      │           577 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 260,545 (1017.75 KB)

 Trainable params: 260,353 (1017.00 KB)

 Non-trainable params: 192 (768.00 B)

In [8]:
model.fit(x_train, y_train, batch_size=8, epochs=10, validation_split=0.2)

NameError: name 'x_train' is not defined

In [ ]:
# Generate dummy test data
x_test = np.random.rand(10, 10, 64, 64, 1)  # 10 samples, 10 timesteps, 64x64 grayscale

# Make predictions
predictions = model.predict(x_test)

# Predictions will have shape (batch_size, height, width, channels)
print(predictions.shape)

In [ ]:
padded_frame = tf.image.resize_with_pad(frame, target_height, target_width)
padded_frame = np.pad(frame, ((0, pad_height), (0, pad_width), (0, 0)), mode='constant')


In [ ]:
# Example of different sizes for each time step
time_series = [
    np.random.rand(32, 32, 1),  # (height=32, width=32, channels=1)
    np.random.rand(64, 64, 1),  # (height=64, width=64, channels=1)
    np.random.rand(48, 48, 1)   # (height=48, width=48, channels=1)
]

# Pad all frames to the maximum size (64x64 here)
max_height, max_width = 64, 64
padded_series = [
    np.pad(frame, ((0, max_height - frame.shape[0]), (0, max_width - frame.shape[1]), (0, 0)), mode='constant')
    for frame in time_series
]

# Stack into a single time-series array
padded_series = np.stack(padded_series)  # Shape: (time_steps, max_height, max_width, channels)


In [ ]:
model = tf.keras.Sequential([
    layers.ConvLSTM2D(
        filters=32, kernel_size=(3, 3), input_shape=(10, 64, 64, 1),
        padding='same', return_sequences=True, activation='relu'
    ),
    layers.BatchNormalization(),  # Normalize outputs from the first ConvLSTM2D
    layers.ConvLSTM2D(
        filters=64, kernel_size=(3, 3),
        padding='same', return_sequences=False, activation='relu'
    ),
    layers.BatchNormalization(),  # Normalize outputs from the second ConvLSTM2D
    layers.Conv2D(
        filters=1, kernel_size=(3, 3),
        activation='sigmoid', padding='same'  # Adjust activation for your task
    )
])


In [ ]:
# Assume you have a trained model and initial data (e.g., X_0)
current_input = X_0  # Initial input (shape: (1, time_steps, height, width, channels))

# Number of steps ahead to predict
num_steps = 3
predictions = []

for step in range(num_steps):
    # Make prediction for the current time step
    prediction = model.predict(current_input)

    # Store the prediction
    predictions.append(prediction)

    # Feed the prediction back into the model for the next step
    # The new input is now the prediction made at this time step
    current_input = prediction

# predictions now contains Y_1, Y_2, Y_3, i.e., the next 3 predicted surfaces
